---
title: "End-to-End Benchmarking"
---

## Processing benchmark results

### Import dependencies

The CarbonPlan team put together some utilities for parsing, processing, and visualizing the benchmarking results in [carbonplan_benchmarks](https://github.com/carbonplan/benchmark-maps). We'll use those utilities along with the [Holoviz](https://holoviz.org/) HoloViz suite of tools for visualization and [Pandas](https://pandas.pydata.org/) as the underlying analysis tool.

In [1]:
import carbonplan_benchmarks.analysis as cba
import hvplot
import holoviews as hv
import pandas as pd

pd.options.plotting.backend = "holoviews"

### Load benchmark results

First, define the paths to the baseline images that the tests will be compared against and paths to the metadata files associated with each benchmarking run.

In [2]:
baseline_fp = "s3://carbonplan-benchmarks/benchmark-data/482049d/baselines.json"
metadata_base_fp = "s3://carbonplan-benchmarks/benchmark-data/"

In [3]:
metadata_files = [
    "207af76/data-2023-08-06T20-16-27.json",
    "75a6745/data-2023-08-06T23-04-38.json",
    "75a6745/data-2023-08-06T22-08-04.json",
    "75a6745/data-2023-08-06T21-32-42.json",
    "4c65e4e/data-2023-08-07T03-07-19.json",
    "4c65e4e/data-2023-08-07T03-16-10.json",
    "4c65e4e/data-2023-08-07T03-34-57.json",
    "4c65e4e/data-2023-08-07T04-04-31.json",
]

Now, use the utilities from `carbonplan_benchmarks` to load the metadata and baseline images into DataFrames, process those results, and create a summary DataFrame for all runs. Given the large size of the trace files, this may take up to several minutes.

In [4]:
snapshots = cba.load_snapshots(snapshot_path=baseline_fp)
summary_dfs = []
nruns = 50
for file in metadata_files:
    fp = f"{metadata_base_fp}/{file}"
    for run in range(nruns):
        metadata, trace_events = cba.load_data(metadata_path=fp, run=run)
        data = cba.process_run(
            metadata=metadata, trace_events=trace_events, snapshots=snapshots
        )
        summary_dfs.append(cba.create_summary(metadata=metadata, data=data))
summary = pd.concat(summary_dfs)

## Visualize results

First, let's see how the duration of each action changes as a function of the zoom level. An important piece of context is that the underlying dataset only has four pyramid levels, so zoom=4 does not need to fetch any new data.

In [5]:
summary[['approach','zarr_version','chunk_size','zoom','duration','fps','request_duration','request_percent']]

,approach,zarr_version,chunk_size,zoom,duration,fps,request_duration,request_percent
0,direct-client,v2,1,0.0,1044.629,54.564826,891.465,85.337953
0,direct-client,v2,1,0.0,965.975,38.303269,825.523,85.460079
0,direct-client,v2,1,0.0,892.225,43.710947,761.602,85.359859
0,direct-client,v2,1,0.0,956.304,36.599240,819.009,85.643164
0,direct-client,v2,1,0.0,876.892,42.194478,756.874,86.313252
...,...,...,...,...,...,...,...,...
0,direct-client,v3,25,0.0,2824.118,27.265150,2559.694,90.636935
0,direct-client,v3,25,0.0,6430.100,7.931447,6177.064,96.064820
0,direct-client,v3,25,0.0,2626.289,20.180567,2367.954,90.163497
0,direct-client,v3,25,0.0,2610.521,17.621004,2366.830,90.665043


In [6]:
cmap=['#E1BE6A', '#40B0A6']
plt_opts={'width': 800, 'height': 600}


In [7]:
summary.hvplot.box(
    y="duration",
    by=["chunk_size", 'zarr_version'],
    c="zarr_version",
    cmap=cmap,
    ylabel="Action duration (ms)",
    xlabel="Chunk size (MB)",
    legend=False
).opts(**plt_opts)


:BoxWhisker   [chunk_size,zarr_version]   (duration)

In [8]:
summary.hvplot.box(
    y="request_duration",
    by=["chunk_size", 'zarr_version'],
    c="zarr_version",
    cmap=cmap,
    ylabel="Time spent on requests (ms)",
    xlabel="Chunk size (MB)",
    legend=False
).opts(**plt_opts)


:BoxWhisker   [chunk_size,zarr_version]   (request_duration)

In [9]:
summary.hvplot.box(
    y="non_request_duration",
    by=["chunk_size", 'zarr_version'],
    c="zarr_version",
    cmap=['#E1BE6A', '#40B0A6'],
    ylabel="Time not spent on requests (ms)",
    xlabel="Chunk size (MB)",
    legend=False
).opts(**plt_opts)

:BoxWhisker   [chunk_size,zarr_version]   (non_request_duration)

In [10]:
summary.hvplot.box(
    y="request_percent",
    by=["chunk_size", 'zarr_version'],
    c="zarr_version",
    cmap=['#E1BE6A', '#40B0A6'],
    ylabel="Percent of time spend on requests",
    xlabel="Chunk size (MB)",
    legend=False
).opts(**plt_opts)

:BoxWhisker   [chunk_size,zarr_version]   (request_percent)

In [11]:
summary.hvplot.box(
    y="fps",by=["chunk_size", 'zarr_version'], c="zarr_version", cmap=cmap, ylabel="Frames per second (FPS)", xlabel="Chunk size (MB)", legend=False
).opts(**plt_opts)


:BoxWhisker   [chunk_size,zarr_version]   (fps)